Importation de tous les packages nécéssaires

In [16]:
import numpy as np
import sys
import os
sys.path.append(os.path.abspath("src"))
from agent import Actor, Critic, ReplayBuffer, DDPGAgent
from env import WirelessEnv
import torch
import rewards_computation
from rewards_computation import compute_rewards

Declaration des différents varibles et objets pour le test

In [17]:
state_dim = 2
response_dim = 1
max_response = 10
Agent = DDPGAgent(state_dim, response_dim, max_response,buffer_size=1000)
env = WirelessEnv(num_antennas=1, num_users=1)

On vérifie si les objets env et Agent sont bien déclarés et à quoi est ce qu'ils ressemblent.

In [18]:
env_state = env._get_state()
print(env_state)
print(Agent)

{'users': [{'id': 0, 'pos': array([35.22288313, 25.0173062 ]), 'demand': 1}], 'antennas': [{'id': 0, 'pos': array([ 0, 50])}]}


In [19]:
demandes = env_state['users'][0]['demand']
print(demandes)

1


 Réponse de Agent à la demande de User. Par convention, la demande est une list ou un ndarray de taille 2 soit la puissance réelle demandée par le user et la distance qui le sépare de l'antenne qui lui est assignée.

In [20]:
distances = env._user_antennas_distance()
print(distances)
# pour user_0
distance_utile = distances[0][0]
print(distance_utile)

{0: {0: 43.18317363431216}}
43.18317363431216


In [21]:
demandes = [demandes, distance_utile]
print(demandes)

[1, 43.18317363431216]


In [22]:
response = Agent.select_response(demandes) # attention la demande ici est une list ou un ndarray
print(response)

[10.]


Calcule de la recompense de l'antenne 0

In [23]:
response_dict = {0: response}
print(response_dict)

{0: array([10.], dtype=float32)}


In [24]:
reward = compute_rewards(response_dict, env)

In [25]:
print(reward)

{0: array([0.2263305], dtype=float32)}


In [26]:
print(demandes, response, reward)

[1, 43.18317363431216] [10.] {0: array([0.2263305], dtype=float32)}


In [27]:
batch_size = 60
for i in range(1000):
    response = Agent.select_response(demandes) 
    response_dict = {0: response}
    reward_dict = compute_rewards(response_dict,env)
    reward = reward_dict[0]
    if i%100 == 0:
        print([demandes, response, reward])
    Agent.buffer.add(demandes, response, reward, demandes)


[[1, 43.18317363431216], array([10.], dtype=float32), array([0.2263305], dtype=float32)]
[[1, 43.18317363431216], array([10.], dtype=float32), array([0.2263305], dtype=float32)]
[[1, 43.18317363431216], array([10.], dtype=float32), array([0.2263305], dtype=float32)]
[[1, 43.18317363431216], array([10.], dtype=float32), array([0.2263305], dtype=float32)]
[[1, 43.18317363431216], array([10.], dtype=float32), array([0.2263305], dtype=float32)]
[[1, 43.18317363431216], array([10.], dtype=float32), array([0.2263305], dtype=float32)]
[[1, 43.18317363431216], array([10.], dtype=float32), array([0.2263305], dtype=float32)]
[[1, 43.18317363431216], array([10.], dtype=float32), array([0.2263305], dtype=float32)]
[[1, 43.18317363431216], array([10.], dtype=float32), array([0.2263305], dtype=float32)]
[[1, 43.18317363431216], array([10.], dtype=float32), array([0.2263305], dtype=float32)]


In [28]:
Agent.buffer.size()

1000

Entrainement

In [29]:
Agent.train()

In [30]:
print(Agent)